# Generate Docs

## Imports and definitions

In [1]:
import pandas as pd
from src.utils import cleanup_folder
from src.tag_doc_page import TagDocPage

In [2]:
output_dir = 'output'
n_questions_per_tag = 5

## Create DataFrames

In [3]:
comments_df = pd.read_csv('data/comments.csv')
posts_questions_df = pd.read_csv('data/posts_questions.csv')
posts_answers_df = pd.read_csv('data/posts_answers.csv')
posts_tag_wiki_df = pd.read_csv('data/posts_tag_wiki.csv')
posts_tag_wiki_excerpt_df = pd.read_csv('data/posts_tag_wiki_excerpt.csv')
selected_tags_df = pd.read_csv('data/selected_tags.csv')
tags_df = pd.read_csv('data/tags.csv')
users_df = pd.read_csv('data/users.csv')

## Development

In [4]:
cleanup_folder(output_dir)

In [5]:
for idx, row in selected_tags_df.iterrows():
    tag_id = row.id
    tag_name = row.tag_name
    tag_description = posts_tag_wiki_excerpt_df.loc[posts_tag_wiki_excerpt_df.id == row.excerpt_post_id]['body'].values[0]
    tag_wiki_body = posts_tag_wiki_df.loc[posts_tag_wiki_df.id == row.wiki_post_id]['body'].values[0]
    TagDocPage(tag_name, tag_description, tag_wiki_body).save(f"{output_dir}/{tag_name}.md")
        